# Setting SNMP alert destination

<div class="alert alert-block alert-info" style="background-color:">
 Welcome to <b>python-iLOrest-library</b> <br> This is an example of <b>setting SNMP alert destination.</b> <br> As you are here let's get it done easily.<br>
    To perform this you need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "background-color:white">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

 <div class="alert alert-block alert-success">
    <h1> 1. Below we are importing all the necessary modules from the ilorest library. </h1>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from ilorest_util import get_resource_directory
from ilorest_util import get_gen

<div class = "alert alert-block alert-success"><h1>2. This is our configure_snmp function overview, in the end this will give us with snmp configure related information.</h1>
    <br>
  There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.

1. In the <b>configure_snmp</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.
<br>
    
2. In the iLO rest api the information about snmp is stored inside the <b>managers json object</b>.
    Through managers_response variable  we are making a http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods" style="background-color:powderblue;">GET operation</a> to and trying to obtain the snmp about iLO system.
    <br>

3. In the managers response object we are iterating on all the members with different <b>odata_id</b> .
    Then again an http GET request is made and response is stored in <b>managers_members_response</b>.
    The snmp configuration information is ultimately present in the <b>managers_members_response</b>.
    <br>

4. Else if resource directory is enabled then we will simply iterate through all instances inside resources 
    and if snmp is present in <b>@odata.type</b> attribute then we will store information about snmp in snmp_service_uri and break out of iteration.
    <br>
Ultimately if snmp_service_uri was obtained without any errors then get the information at the relevant uri and print out that information .

 </div>

In [ ]:
def configure_snmp(_redfishobj, read_communities, snmp_alertdestinations, DISABLE_RESOURCE_DIR):
    snmp_service_uri = None
    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        snmp_service_uri = managers_members_response.obj.Oem.Hpe.Links['Snmp']['@odata.id']
    else:
        for instance in resource_instances:
            #Use Resource directory to find the relevant URI
            if '#HpeiLOSnmpService.' in instance['@odata.type']:
                snmp_service_uri = instance['@odata.id']

    if snmp_service_uri:
        #body = {"AlertsEnabled": snmp_alerts, "ReadCommunities": read_communities}
        body = {"AlertDestinations": snmp_alertdestinations}
        resp = _redfishobj.patch(snmp_service_uri, body)

        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                                                                                    sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                                     "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success"><h1>3. This is our set_snmp_alert_destination function overview. </h1>
    <br>
1. The <b>set_snmp_alert_destination</b> function sets the SNMP alert destinations on an iLO device. The function takes three parameters: _redfishobj, snmp_service_uri, and alert_destination_list. 
    <ul>
    <li><b>_redfishobj</b> is the Redfish client object.</li>
    <li><b>snmp_service_uri</b> is the URI for the SNMP service on the iLO device.</li>
    <li><b>alert_destination_list</b> is a list of dictionaries containing the SNMP alert destinations to set.</li>
    </ul><br>
2. The function gets the current SNMP service data using <b>_redfishobj.get()</b> and checks if the <b>AlertDestinations</b> property is available/modifyable.<br><br>
3. If the property is available/modifyable, the function sends a PATCH request with the updated alert destinations using <b>_redfishobj.patch()</b>.<br><br> 
4. If the iLO device responds with a status code outside of 200 or 201, the function attempts to access the iLO extended message info to see what went wrong.<br><br>   
5. If the patch operation is successful, the function prints the response and gets the updated SNMP alert destination data using <b>_redfishobj.get()</b>.<br><br>   
6. The updated SNMP alert destination data is printed to the console using <b>json.dumps()</b> and <b>print()</b>.<br><br>
 </div>

In [ ]:
def set_snmp_alert_destination(_redfishobj, snmp_service_uri, alert_destination_list):
    data = _redfishobj.get(snmp_service_uri)
    if data.dict.get("AlertDestinations"):
        resp = _redfishobj.patch(snmp_service_uri, {'AlertDestinations': alert_destination_list})
    else:
        raise Exception("\'AlertDestinations\' property is not available/modifyable.\n")
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended " "Message Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("\nPatch operation successful!\n\nResponse:")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))
        snmp_service_response = _redfishobj.get(snmp_service_uri).dict.get('AlertDestinations')
        print("\n\nPrinting updated SNMP alert destination:\n")
        print(json.dumps(snmp_service_response, indent=4, sort_keys=True))


<div class="alert alert-block alert-success"><h1 >4. This is Main function overview.</h1><br>
<b>The Main function </b>
    <ul>
        <li>Once your credentials that are enter everything a redfish object is created using the input system URL, login account, and login password with the help of RedfishClient class and a session is created and now you will be logged in to the server.</li>
        <li>If server is down or unreachable we get server not reachable message.</li>
        <li>It checks the generation of iLO by calling the get_gen function and prints the iLO generation.
        <li>If the iLO generation is 5, it calls the <b>configure_snmp</b> function to configure the SNMP service with the read communities and alerts destination, and disable resource directory flag.</li>
<li>If the iLO generation is not 5, it calls the <b>set_snmp_alert_destination</b> function to set the SNMP alert destination with the alert destination list for iLO4.</li>
        <li>After getting required information <i><b>Redfish.logout()</b></i> closses the particular session and logs us out of the system.</li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    #SYSTEM_URL = "https://15.146.46.45"
    #LOGIN_ACCOUNT = "admin"
    #LOGIN_PASSWORD = "admin123"

    SYSTEM_URL = sys.argv[1]
    LOGIN_ACCOUNT = sys.argv[2]
    LOGIN_PASSWORD = sys.argv[3]

    #Properties:
    #read communities array
    READ_COMMUNITIES = ["public", "", ""]
    #alerts_enabled primitive (boolean)
    #ALERTS_ENABLED = True
    #Alert Destination
    ALERTS_DESTINATION = ["1.1.1.1","2.2.2.2"]
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False
    snmp_service_uri = "/redfish/v1/Managers/1/SNMPService/"

    # Number of max alert destination supported on iLO4 and iLO5 is different
    alert_destination_list = ["ILOCN771702NJ", "15.146.46.55" , "15.146.46.58"]

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()
    (ilogen,_) = get_gen(REDFISHOBJ)
    print ("Generation is ", ilogen)
    if int(ilogen) == 5:
        configure_snmp(REDFISHOBJ, READ_COMMUNITIES, ALERTS_DESTINATION, DISABLE_RESOURCE_DIR)
    else:
        set_snmp_alert_destination(REDFISHOBJ, snmp_service_uri, alert_destination_list)
    REDFISHOBJ.logout()


<div class = "alert alert-block alert-warning">
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>